In [2]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
import os 
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOING']='https://api.smith.langchain.com'

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Multi Query
Index

In [5]:
### Indexing ###
# Load blog
import bs4 # BeautifulSoup will help to provide you clean content from webpage
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs=loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)

# Make split
splits=text_splitter.split_documents(blog_docs)

# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore=Chroma.from_documents(documents=splits,embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'))
retriever=vectorstore.as_retriever()

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`